# Introduction

The idea behind nornir_tests is to allow for adding tests against the task run and its results.  Many times, a task can run and get some output but that doesn't mean it is successful.  One option is to add additional checks or tasks with some conditional logic to deal with this.  Another is to add tests against the data returned from a task.  That is what nornir_tests is for.

The question is probably why use tests.  Tests were inspired by the same feature in postman.  In postman, a test was something that was done against the data returned from a postman request to determine if the request was actually successful.  This can certainly be added in python using either pure python or additional tasks but there is some elegance and some additional functionality obtained by using tests tied to tasks.  In this intro, I will try to touch on a variety of ideas using tests.  In further tutorials I'll go into more detail on using each of the test types.

## Nornir initialization

This is standard nornir initialization and imports of tasks.  A different library nornir_rich is used for output to provide more detail and highlighting.  Using print_result is also possible if vars is set to include 'tests' attribute.

In [1]:
from nornir_napalm.plugins.tasks import napalm_get, napalm_ping
from nornir_netmiko.tasks import netmiko_send_command
from nornir_rich.plugins.functions import RichResults
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)

rr = RichResults()

## Setup of nornir_rich

TestsProcessor is appended to the processors list and the inventory is shown. This can be done using config.yaml as well or a few other ways that are covered in the Nornir documentation.

In [2]:
from nornir_tests.plugins.tests import test_regexp, test_until, test_timing, test_jsonpath
from nornir_tests.plugins.processors import TestsProcessor
nr.processors.append(TestsProcessor())

## Run task and include jsonpath assertion

In the first example, a vyos router interface facts are obtained using napalm and a test is run to validate the result.

In [4]:
vyos = nr.filter(name='vyos')

result = vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_jsonpath(path='interfaces.eth0.is_up', assertion='is_true'),
    ]
)

rr.print(result, vars=['result', 'tests'])

## Run ping with timing requirements

Next a ping is sent to the vyos device using the timing test.  As can be seen in the output, this gives runtime of the task and can be enhanced with min_run_time and max_run_time to affect the task result if the fail_task argument is used.  If a failure occurs in the test and fail_task is set to True, then the task will be marked as failed.

In [6]:
result = vyos.run(
    task=napalm_ping,
    dest='192.168.99.1',
    tests=[
        test_timing(),
    ]
)
rr.print(result, vars=['result', 'tests'])

## Get unfailed task to fail due to assertion

The next example shows how the result failed state can be affected by a test.

In [7]:
result = vyos.run(
    task=napalm_ping,
    dest='192.168.99.1',
    tests=[
        test_timing(max_run_time=1, fail_task=True),
    ]
)
rr.print(result, vars=['result', 'tests'])

## Combining tests

Tests can be stacked and the entire task and associated tests can be retried using test_until.  The next test shows how a rebooted device can be retried until it comes back up using only tests.

In [9]:
nr.data.reset_failed_hosts()

rr.print(
    vyos.run(
        task=netmiko_send_command,
        command_string='reboot now'
    )
)

result = vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_jsonpath(path='interfaces.eth0.is_up', assertion='is_true', fail_task=True),
        test_until(initial_delay=15, retries=10, delay=15, reset_conns=True),
    ]
)
rr.print(result, vars=['result', 'tests'])

Not bad.  The result above shows that it must have failed on its first try but by 36 seconds it was able to pass the test_jsonpath test and the call to napalm_get and so test_until finished up.

This was pretty haphazard but other tutorials will go into more details about each of the tests.